In [1]:
import pandas as pd
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
import numpy as np
from torch import nn
from torch.utils.data import DataLoader
import torch
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
def metric(true, pred, coef=1000):
    assert len(true) == len(pred)
    error = 0
    for i in range(len(true)):
        if true[i] > pred[i]:
            error += true[i] - pred[i]
        else:
            error += (pred[i] - true[i]) * coef
    return -round(error / len(true) / 1_000_000_000)

def metric_loss(true, pred, coef=1000.0):
    assert len(true) == len(pred)

    diff = true - pred
    loss = torch.abs(diff)
    loss = torch.where(diff < 0, loss * coef, loss)
    return loss.mean()

In [3]:
class CustomLoss(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, pred, true, coef=1000.0):
        assert len(true) == len(pred)

        diff = true - pred
        loss = torch.abs(diff)
        loss = torch.where(diff < 0, loss * coef, loss)
        return loss.mean()
    
class CustomBCELoss(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, output, target):        
        sigm = nn.functional.sigmoid(output)
        target = nn.functional.sigmoid(target)
        left = target * torch.log(sigm)
        right = (1 - target) * torch.log(1 - sigm)
        print(sigm)
        print(target)
        
        loss = left + right
        return -loss.mean()

In [4]:
df_train = pd.read_csv('data/train.csv')
df_test = pd.read_csv('data/test.csv')

In [5]:
df_train

,id,day 1,day 2,day 3,day 4,day 5,day 6,day 7,day 8,day 9,...,day 81,day 82,day 83,day 84,day 85,day 86,day 87,day 88,day 89,day 90
0,6602185159948,569369144437,152818843344,-15490421929,-406624212,800362695678,2306966308406,1917859437929,1075925676467,277630770065,...,-23536698564,1524999469315,1668473296482,-55911238122,1105906678699,1134120612066,-26829699967,-19492512239,1850461161279,-1781621104800
1,1663310387493,-104722473995,-29275417695,423353969042,416864670779,-323866168258,-59323097843,58355171506,250838843414,-184607036341,...,529867756317,-245569015020,-13050008306,1122739774405,-358972410001,-294465761456,289760186052,588830896355,-21661542192,-153747680862
2,3604676372924,-42697753432,154940888256,5290701628,2915610080,-241412656598,-2172755647,-60115526657,99210264232,281086593365,...,32773187167,-347787851371,172520154368,46099833867,-297776587790,200380940252,153279754373,42773997259,5607112117,4789271560371
3,3517792576177,735945593435,877636042341,-5241075542,-1817826687,1008399822616,996548931508,471592893006,479393037398,246618026800,...,4227540489,855279732927,808871471815,-8997018250,627399608353,223093070252,-25844611162,-9955925825,603978495789,366344015281
4,6892195368446,697850269572,980815916948,-11470196952,-21103754202,1207799272628,1202816134496,559646086486,1058635690664,122661998719,...,-17229675906,940502608045,728437869463,13477840270,1455297859610,409612399843,-6730004331,19193466540,1163355637082,-138405088616
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,6120398522001,-24628787635,267495848867,-12230119509,-679136157,-187955674883,4597158234,-2418401538,9854043544,-14559914953,...,-7158916464,204729538824,-39577541831,-18068542326,105241442152,-59936288497,-26435699898,-1400118718,250752357103,818156843427
19996,3883376591531,23385081422,-15228303151,-8918281087,-7772984843,-7543131908,123891713099,-81767137062,62477678253,44174672734,...,-7626731747,157231720804,79682589968,-26953658290,53427137426,59564486558,-20909395485,-15970211056,223711674873,-815055027
19997,8492364329502,-17572384426,215467299511,-9758708941,-3626522770,-182333216840,70907176369,-16418782963,-1289752268,299779480157,...,-17792945695,-336100103876,266432384316,-36450926686,-7151728550,185309970484,59247462995,-17951159898,-66380991478,3395054610973
19998,4485476161662,-34373226151,-123943635780,2186315945,1323094644,44423732764,67484356011,-29553565822,66448231612,-3151812187,...,29253821343,186004596897,-38862536845,49943281842,-70544749716,106782921385,51233576000,43731331086,-3692332109,-89024228031


In [6]:
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [7]:
device = 'cuda'

In [8]:
class Net(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_dim * 2, output_dim, bias=True)
        
    def forward(self, x):
        h0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_dim).requires_grad_()
        c0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_dim).requires_grad_()
        out, (hn, cn) = self.lstm(x, (h0.detach().to(device), c0.detach().to(device)))
        out = self.fc(out) 
        return out

In [9]:
model = Net(89, 256, 1, 1)
model.to(device).train()

Net(
  (lstm): LSTM(89, 256, batch_first=True, bidirectional=True)
  (fc): Linear(in_features=512, out_features=1, bias=True)
)

In [10]:
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-3)

In [11]:
min_loss = 10e100
loss_history = []
metric_history = []

In [12]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scal = StandardScaler()

In [13]:
data = []
data_v = []
data_t = []

new_def_train = scaler.fit_transform(df_train[:15000].drop('day 90', axis=1)[[f'day {x}' for x in range(1, 90)]].to_numpy())
new_train_y = scal.fit_transform(df_train[:15000]['day 90'].to_numpy().reshape(-1, 1))

new_def_val = scaler.transform(df_train[15000:].drop('day 90', axis=1)[[f'day {x}' for x in range(1, 90)]].to_numpy())
new_val_y = df_train[15000:]['day 90'].to_numpy().reshape(-1, 1)

new_def_test = scaler.transform(df_test[[f'day {x}' for x in range(1, 90)]].to_numpy())

for value, y in zip(new_def_train, new_train_y):
    data.append((value).tolist() + [y])
    
for value, y in zip(new_def_val, new_val_y):
    data_v.append((value).tolist() + [y])
    
for value in scaler.transform(new_def_test):
    data_t.append(value.tolist())

In [14]:
len(data_t[0])

89

In [15]:
criterion = CustomLoss()

In [16]:
best_score = -10e100
best_epoch = 0

In [17]:
for epoch in tqdm(range(500)):
    np.random.shuffle(data)
    train_x = torch.tensor([i[:-1] for i in data])
    train_y = torch.tensor([i[-1] for i in data])
    x_loader = DataLoader(train_x, batch_size=len(train_x))
    y_loader = DataLoader(train_y, batch_size=len(train_y))

    for x, y in zip(x_loader, y_loader):
        output = model(x.view(1, *x.shape).float().to(device))

        loss = criterion(output.view(-1), y.view(-1).float().to(device))
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        del x, y
        torch.cuda.empty_cache()
        
        current_loss = round(loss.item(), 4)
        loss_history.append(current_loss)
        
        if current_loss < min_loss:
            min_loss = current_loss

        loss_data = {
            'loss_history': loss_history
        }

        dataframe = pd.DataFrame(loss_data)
        plt.ioff()
        fig, ax = plt.subplots()
        sns.lineplot(data=dataframe, ax=ax)
        plt.title(f'Net loss, min = {min_loss}')
        ax.set(xlabel='batches', ylabel='loss')
        fig.savefig('net_loss.png')
        plt.close()
        
        val_x = torch.tensor([i[:-1] for i in data_v]).to(torch.float32)
        val_y = [i[-1][0] for i in data_v]

        output = scal.inverse_transform(model(val_x.view(1, *val_x.shape).to(device)).view(-1).reshape(1, -1).detach().cpu()).reshape(-1).tolist()
        metric_data = metric(val_y, list(map(lambda x: int(x * 1), output)))
        
        if metric_data > best_score:
            best_score = metric_data
            best_epoch = epoch
        
        metric_history.append(metric_data)
        
        metric_data = {
            'metric_history': metric_history[5:]
        }
        
        dataframe = pd.DataFrame(metric_data)
        plt.ioff()
        fig, ax = plt.subplots()
        sns.lineplot(data=dataframe, ax=ax)
        plt.title(f'Net metric, min = {best_score}')
        ax.set(xlabel='batches', ylabel='metric')
        fig.savefig('net_metric.png')
        plt.close()

  0%|                                                                                          | 0/500 [00:00<?, ?it/s]C:\ProgramData\Anaconda3\envs\nto_final\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:253.)
  after removing the cwd from sys.path.
100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [05:56<00:00,  1.40it/s]


In [18]:
best_score

-661

In [19]:
best_epoch

460

In [20]:
model.eval()

Net(
  (lstm): LSTM(89, 256, batch_first=True, bidirectional=True)
  (fc): Linear(in_features=512, out_features=1, bias=True)
)

In [21]:
val_x = torch.tensor([i[:-1] for i in data_v]).to(torch.float32)
val_y = [i[-1][0] for i in data_v]

output = scal.inverse_transform(model(val_x.view(1, *val_x.shape).to(device)).view(-1).reshape(1, -1).detach().cpu()).reshape(-1).tolist()
metric_data = metric(val_y, list(map(lambda x: int(x * 1), output)))

In [22]:
metric_data

-690

In [23]:
len(list(filter(lambda x: x < 0, output)))

3215

In [24]:
test_x = torch.tensor([i for i in data_t]).to(torch.float32)

output_t = scal.inverse_transform(model(test_x.view(1, *test_x.shape).cuda()).view(-1).reshape(1, -1).detach().cpu()).reshape(-1).tolist()
output_t = list(map(lambda x: x * 1, output_t))

In [25]:
answer = pd.DataFrame({'id': df_test['id'].tolist(), 'day 91': map(int, output_t)})
answer.to_csv('submission.csv', index=False)

In [26]:
len(list(filter(lambda x: x < 0, output_t)))

9999